In [1]:
import numpy as np
import pandas as pd
from skimage import transform
from keras.models import Model, Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, GlobalAveragePooling2D, GlobalMaxPooling2D
from keras import losses, optimizers, callbacks
from keras.applications import vgg19, inception_v3
import xgboost as xgb
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate, train_test_split
from scipy import fftpack
import matplotlib.pyplot as plt
%matplotlib inline
RANDOM_SEED = 43
np.random.seed(RANDOM_SEED)

Using TensorFlow backend.
/home/can/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Create Dataset

In [2]:
# Train
output_shape = (300, 300)
df = pd.read_json('./data/train.json')
X_train, y_train = [], []
for im_band1, im_band2, label in zip(df['band_1'], df['band_2'], df['is_iceberg']):
    im_band1 = np.array(im_band1).reshape(75, 75, 1)
    im_band2 = np.array(im_band2).reshape(75, 75, 1)  
    im_band1_fft = fftpack.fftshift(np.log(np.abs(fftpack.fft2(im_band1))))
    im_bands_avg = (im_band1 + im_band2) / 2.0
    # Preprocess - resize
    im_band1 = transform.resize(im_band1, output_shape=output_shape)
    im_band2 = transform.resize(im_band2, output_shape=output_shape)
    im_bands_avg = transform.resize(im_bands_avg, output_shape=output_shape)
    # Preprocess - normalize
    im_band1 /= np.std(im_band1)
    im_band2 /= np.std(im_band2)
    im_bands_avg /= np.std(im_bands_avg)
    im_band1_fft /= np.std(im_band1_fft)
    # Concatenate
    im = np.concatenate([im_band1, im_band2, im_bands_avg], axis=2)
    X_train.append(im)
    y_train.append(label)    
X_train = np.array(X_train)
y_train = np.array(y_train)
print 'X_train.shape:', X_train.shape
print 'y_train.shape:', y_train.shape

/home/can/anaconda2/lib/python2.7/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


X_train.shape: (1604, 300, 300, 3)
y_train.shape: (1604,)


# Train - Val Split

In [3]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=RANDOM_SEED)
print 'X_train.shape:', X_train.shape
print 'X_val.shape:', X_val.shape
print 'y_train.shape:', y_train.shape
print 'y_val.shape:', y_val.shape

X_train.shape: (1283, 300, 300, 3)
X_val.shape: (321, 300, 300, 3)
y_train.shape: (1283,)
y_val.shape: (321,)


# Data Augmentation

In [4]:
def bypass(x):
    return x

def h_flip(x):
    return x[:, :, ::-1, :]

def v_flip(x):
    return x[:, ::-1, :, :]

def hv_flip(x):
    return h_flip(v_flip(x))

def rot90(x):
    return np.concatenate([np.expand_dims(transform.rotate(im, 90), axis=0) for im in x], axis=0)

def rot180(x):
    return np.concatenate([np.expand_dims(transform.rotate(im, 180), axis=0) for im in x], axis=0)

def rot270(x):
    return np.concatenate([np.expand_dims(transform.rotate(im, 270), axis=0) for im in x], axis=0)

def rot45(x):
    return np.concatenate([np.expand_dims(transform.rotate(im, 45, mode='reflect'), axis=0) for im in x], axis=0)

def rot135(x):
    return np.concatenate([np.expand_dims(transform.rotate(im, 135, mode='reflect'), axis=0) for im in x], axis=0)

def rot315(x):
    return np.concatenate([np.expand_dims(transform.rotate(im, 315, mode='reflect'), axis=0) for im in x], axis=0)

aug_funcs = [bypass, 
             h_flip, v_flip, hv_flip,
             rot90, rot180, rot270]

In [5]:
# Train
X_train = np.concatenate([func(X_train) for func in aug_funcs], axis=0)
y_train = np.concatenate([y_train] * len(aug_funcs))

# Validation
X_val = np.concatenate([func(X_val) for func in aug_funcs], axis=0)
y_val = np.concatenate([y_val] * len(aug_funcs))

# 
print 'X_train.shape:', X_train.shape
print 'y_train.shape:', y_train.shape
print 'X_val.shape:', X_val.shape
print 'y_val.shape:', y_val.shape

X_train.shape: (8981, 300, 300, 3)
y_train.shape: (8981,)
X_val.shape: (2247, 300, 300, 3)
y_val.shape: (2247,)


# Training

In [8]:
# Make model
INPUT_SHAPE = (300, 300, 3)
model = inception_v3.InceptionV3(weights=None, classes=1)

In [9]:
# Compile
loss = losses.binary_crossentropy
optimizer = optimizers.Adam()
metrics = ['accuracy']
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

# Fit
MODEL_PATH = './models/model3.h5'
m_q = 'val_loss'
check_pt = callbacks.ModelCheckpoint(filepath=MODEL_PATH, monitor=m_q, save_best_only=True, verbose=1)
early_stop = callbacks.EarlyStopping(patience=3, monitor=m_q, verbose=1)
reduce_lr = callbacks.ReduceLROnPlateau(patience=1, factor=0.5, monitor=m_q, verbose=1)
callback_list = [check_pt, early_stop, reduce_lr]

model.fit(X_train, y_train, validation_data=(X_val, y_val), callbacks=callback_list, epochs=200)

Train on 8981 samples, validate on 2247 samples
Epoch 1/200
4896/8981 [===============>..............] - ETA: 57s - loss: 8.5215 - acc: 0.4655

KeyboardInterrupt: 

In [ ]:
# Load the model
model = load_model(MODEL_PATH)

# -- Entire Model Train --
for layer in model.layers:
    layer.trainable = True

# Compile
loss = losses.binary_crossentropy
optimizer = optimizers.SGD(lr=1e-4)
metrics = ['accuracy']
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

# Fit
m_q = 'val_loss'
check_pt = callbacks.ModelCheckpoint(filepath=MODEL_PATH, monitor=m_q, save_best_only=True, verbose=1)
early_stop = callbacks.EarlyStopping(patience=6, monitor=m_q, verbose=1)
reduce_lr = callbacks.ReduceLROnPlateau(patience=1, factor=0.5, monitor=m_q, verbose=1)
callback_list = [check_pt, early_stop, reduce_lr]

model.fit(X_train, y_train, validation_data=(X_val, y_val), callbacks=callback_list, epochs=200)

Train on 8981 samples, validate on 2247 samples
Epoch 1/200
8981/8981 [==============================] - 139s - loss: 0.2993 - acc: 0.8600 - val_loss: 0.3135 - val_acc: 0.8505
Epoch 2/200
8981/8981 [==============================] - 126s - loss: 0.2907 - acc: 0.8692 - val_loss: 0.3091 - val_acc: 0.8456
Epoch 3/200
8981/8981 [==============================] - 126s - loss: 0.2787 - acc: 0.8743 - val_loss: 0.2976 - val_acc: 0.8540
Epoch 4/200
8960/8981 [============================>.] - ETA: 0s - loss: 0.2674 - acc: 0.8810